In [ ]:
##### prepare 10k IMMU tables 
# Step 1: investigate single studies for data collection platforms
# Step 2: Describe “formatted” preprocessing way as i can find in literature -> for Microarray its rather clear, right ?
# Step 3: Standartize them to make them comparable
# Step 4: Add 3-4 paper´s data to also get a deseased dataset (or maybe even finish the search in immport, as I think new studies got collected since release of 10kimmu)
# Step 5: GAMLLS model creation. 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys
sys.path.append('/home/eliasschreiner/PROJEKT/utils/')  # Adjust this to the path where `utils` resides

from utils.helper import manhattanplot, violinplot_overall, scatter_plot, get_negative_values, METADATA_COLS


In [ ]:

ten_k_prefix="/var/datasets/10kImmunomes/transcriptomics/"
### Files we got:
## pbmc_rna_counts.csv
## whole_blood_rna_tpm.csv
## Gene_Expression_PBMC 
### gene_pbmc_formatted.csv ; gene_pbmc_normalized.csv  -> i take formatted as the wholeblood only exists in formatted... 
## Gene_Expression_Whole_Blood_formatted.csv 

print("\n###########################################################")
print("rna_seq_whole_blood.csv")
print("============================================================")
rna_seq_whole_blood = pd.read_csv(f"{ten_k_prefix}/whole_blood_rna_tpm.csv",sep=",")
print(rna_seq_whole_blood.columns)
print(rna_seq_whole_blood.shape)

print("\n###########################################################")
print("marray_whole_blood")
print("============================================================")
marray_whole_blood = pd.read_csv(f"{ten_k_prefix}/Gene_Expression_Whole_Blood_formatted.csv",sep=",")
print(marray_whole_blood.columns)
print(marray_whole_blood.shape)


# Microarray

In [ ]:
####Microarray Data: Gene expression array data were obtained in three formats.
# Data in their original formats (.CEL files, series matrix files, etc) constitute the “raw” gene expression data.

# For data collected on Affymetrix platforms, we utilized the ReadAffy utility in the affy Bioconductor package to read in raw .CEL files. 
# The rma utility was used to conduct Robust Multichip Average (rma) background correction (as in (4)), quantile normalization, and log2 normalization of the data. 
##### CRITERIA: PLATFORM_DESCRIPTION contains 'Affymetrix'

# For data collected on Illumina platforms and stored in the Gene Expression Omnibus (GEO) database, we utilized the getGEO utility in the 
# GEOquery Bioconductor package to read the expression files and the preprocessCore package to conduction rma background correction, quantile normalization, and log2 normalization of the gene expression data.
##### CRITERIA: PLATFORM_DESCRIPTION contains 'illumina'

# Finally, for data collected on Illumina platforms but not stored in GEO, we utilized the read.ilmn utility of the limma Bioconductor package 
# to read in the data, and the neqc function to rma background correct, quantile normalize, and log2 normalize the gene expression data.
##### CRITERIA: Study has GSMxxxxxxxx information for study ? 

# In all instances, probe IDs were converted to Entrez Gene IDs. Where multiple probes mapped to the same Entrez Gene ID, 
# the median value across probes was used to represent the expression value of the corresponding gene.
# The background-corrected and normalized datasets were combined based on common Entrez IDs, missing values were imputed with a 
# k-nearest neighbors algorithm (R package: impute, function: impute.knn) using k = 10 and default values for rowmax, colmax, and maxp. 
# Enter Gene IDs were then converted to HUGO gene names. 
# These steps together produced the “formatted” gene expression files.

# To create the “formatted and normalized” datasets, we utilized established empirical Bayes algorithm ComBat for batch correction (2), compensating for possible batch effects while maintaining potential effects of age, race, and sex across datasets.


In [ ]:

#### Procedure of getting the platform: 
#### Try to get the Repo_Accessions (gene_expression table) for the study.
######## If found, output all platforms for the accessions. (was always only 1)
######## If not found, download the protocols from ImmPort via study acession -> read the microarray protocol and try to find the geo id (succeeded in all but SDY364) 
# needed: immport_platform.ipynb
marray_whole_blood = pd.read_csv(f"{ten_k_prefix}/Gene_Expression_Whole_Blood_formatted.csv",sep=",")
mapping_platform_geo_id = {'SDY111': 'GPL10558', 'SDY112': 'GPL15207', 
                    'SDY113': 'GPL15207', 'SDY144': 'GPL10558',
                    'SDY180': 'GPL6947', 'SDY212': 'GPL6947',
                    'SDY301': 'GPL6947', 'SDY305': 'GPL15207',
                    'SDY364': '?', 'SDY690': 'GPL6947',
                    }	
mapping_platform_description = {'SDY111': 'Illumina HumanHT-12 V4.0 expression beadchip', 'SDY112': '[PrimeView] Affymetrix Human Gene Expression Array', 
                    'SDY113': '[PrimeView] Affymetrix Human Gene Expression Array', 'SDY144': 'Illumina HumanHT-12 V4.0 expression beadchip',
                    'SDY180': 'Illumina HumanHT-12 V3.0 expression beadchip', 'SDY212': 'Illumina HumanHT-12 V3.0 expression beadchip',
                    'SDY301': 'Illumina HumanHT-12 V3.0 expression beadchip', 'SDY305': '[PrimeView] Affymetrix Human Gene Expression Array',
                    'SDY364': 'Illumina Human WG-6 V4 beadchips', 'SDY690': 'Illumina HumanHT-12 V3.0 expression beadchip',
                    }
marray_whole_blood['PLATFORM_GEO_ID'] = marray_whole_blood['study_accession'].apply(lambda x: mapping_platform_geo_id.get(x))
marray_whole_blood['PLATFORM_DESCRIPTION'] = marray_whole_blood['study_accession'].apply(lambda x: mapping_platform_description.get(x))


### For SDY112: data_Accession = Ji_Batch2_22.295185 ...   -> download protocols hat geholfen
### SDY301 -> ES769620 aber finde nicht im expsample ...  -> nicht in gene_expression ABER in expsample 
###### in EXPSAMPLE table: Upload_status -> Not parsed. also ich denke es ist irgendwas kaputt gegangen, es hat auch nur einen eintrag .  aus Experiment id: 'AIRFV Microarray'  -> download protocols hat geholfen
### SDY690 -> ES769464 ... same story...  wieso sind die nicht im gene expression enthalten ? 
###### Exp name: 'Compare whole blood (Tempus tube) to PBMC gene...	' -> download protocols hat geholfen
### SDY364 -> 8962917052_E ?????? wat  -> download protocols hat geholfen 
###### Leider hab ich die GEO accession nicht komplett wiedergefunden und hab mal GPL14951 eingetragen

## Rearrange columns: 
cols = marray_whole_blood.columns.tolist()
cols = cols[-2:] + cols[:-2]
marray_whole_blood = marray_whole_blood[cols]

print(marray_whole_blood['study_accession'].unique()) 
marray_whole_blood = marray_whole_blood.dropna()
marray_whole_blood.to_csv(f'whole_blood/microarray_whole_blood/marray_whole_blood_prepared.csv')

print(f'Studies: {marray_whole_blood.shape}')
print(f'Studies: {len(marray_whole_blood['study_accession'].unique())}')
print(f'Subjects: {len(marray_whole_blood['subject_accession'].unique())}')
print(f'Data ACCESSIONS ??: {len(marray_whole_blood['data_accession'].unique())}')
marray_whole_blood.head()


In [ ]:
marray_whole_blood.describe()

## Error: Negative values of Study 180 (Microarray whole blood)... I spotted that neg values is a microarray thing to do ... i need to standardize this differently. ... 
### remove SDY180

In [ ]:
#marray_whole_blood[marray_whole_blood['study_accession'] == 'SDY180'].shape
marray_whole_blood = marray_whole_blood[~marray_whole_blood.apply(lambda row: row.astype(str).str.contains('SDY180').any(), axis=1)]
marray_whole_blood.shape

In [ ]:
marray_whole_blood_wo_meta = marray_whole_blood.iloc[:, 8:]
marray_whole_blood_wo_meta = marray_whole_blood_wo_meta.reindex(sorted(marray_whole_blood_wo_meta.columns), axis=1)
marray_whole_blood_wo_meta.head()

In [ ]:
marray_whole_blood.describe()

In [ ]:
get_negative_values(marray_whole_blood)

In [ ]:
marray_whole_blood_young = marray_whole_blood[marray_whole_blood['age']<20]
manhattanplot(marray_whole_blood_young, save_name='whole_blood/microarray_whole_blood')

In [ ]:
violinplot_overall(marray_whole_blood, x_topic='gender', y_topic='age', hue_split='gender', bin_name='whole_blood/microarray_whole_blood')
# starke abweichung von RNAseq, ComBat für bayes based batch correction -> soll wohl lt. homas nicht weil GAMLSS das bereits berücksichtigt


# ################ RNAseq

In [ ]:
#### RNA-Seq Whole Blood and PBMC Data: 
# The RNA-Seq whole blood data was downloaded from Immport (Studies: SDY1092, SDY1172, SDY1381, and SDY1412).
# The files used from Immport were formatted and normalized by the EPIC Consortium. 
#### This apparently made the FKPM transofrmation. -> TPM
# The whole blood datasets were converted into TPM files and harmonized together using the HarmonyRNA algorithm. 
#### 
# More information on the study can be found in this Nature Article published by Amy H. Lee, et al. 
# Some minor additional formatting was done for presenting data on 10k Immunomes.

In [ ]:

## Formatted means: background correction, quantile normalization, and log2 normalization
## Normalized: empirical Bayes algorithm for batch correction (2), compensating for possible batch effects

print('RNAseq Whole blood: ')
rna_seq_whole_blood_T = rna_seq_whole_blood.T
rna_seq_whole_blood_T.reset_index(0, inplace=True)

## Hier hab ich quasi 2 headlines: eins ist eben für die ENSGxxxxx nummern und eine f+r den Gene bezeichner -> mapping table 
ensg_numbers = rna_seq_whole_blood_T.iloc[0]
ensg_2_gene_mapping_df = pd.DataFrame([rna_seq_whole_blood_T.iloc[0], rna_seq_whole_blood_T.iloc[1]]).T
ensg_2_gene_mapping_df.columns=ensg_2_gene_mapping_df.iloc[0]
ensg_2_gene_mapping_df = ensg_2_gene_mapping_df.drop(ensg_2_gene_mapping_df.index[0])


rna_seq_whole_blood_T.columns = rna_seq_whole_blood_T.iloc[1]
rna_seq_whole_blood_T = rna_seq_whole_blood_T.drop(rna_seq_whole_blood_T.index[0])
rna_seq_whole_blood_T = rna_seq_whole_blood_T.drop(rna_seq_whole_blood_T.index[0])

### METADATA
rnaseq_whole_blood_metadata = pd.read_csv(ten_k_prefix+'rnaseq_whole_blood_metadata.csv')
print(rnaseq_whole_blood_metadata.shape)

### Bugfix for SDY1381 Subject names
def add_X(row):
    if row['Study']=='SDY1381':
        return 'X' + row['Subject']
    else:
        return row['Subject']
rnaseq_whole_blood_metadata['Subject'] = rnaseq_whole_blood_metadata.apply(add_X, axis=1)

print(rna_seq_whole_blood_T.shape)
rna_seq_whole_blood_T = rna_seq_whole_blood_T.rename(columns={'hgnc': 'Subject'})
rna_seq_whole_blood_T_w_meta = pd.merge(rna_seq_whole_blood_T, rnaseq_whole_blood_metadata, on='Subject', how='inner')

## da wir hier glaub dier selben subjects mit longitudinal data haben: 
unique_subjects = [x.split('_')[0] for x in rna_seq_whole_blood_T_w_meta['Subject'].tolist()]
print(f'Genes: {rna_seq_whole_blood_T_w_meta.shape}')
print(f'Studies: {len(rna_seq_whole_blood_T_w_meta['Study'].unique())}')
print(rna_seq_whole_blood_T_w_meta['Study'].unique())
print(f'Subjects: {len(set(unique_subjects))}')

mapping_platform_geo_id = {'SDY1412': 'GPL16791', 
                           'SDY1092': 'GPL11154', 
                           'SDY1172': 'GPL11154', 
                           'SDY1381': 'GPL11154'}
mapping_platform_description = {'SDY1412': 'Illumina HiSeq 2500 (Homo sapiens)', 'SDY1092': 'Illumina HiSeq 2500 (Homo sapiens)', 
                    'SDY1172': 'Illumina HiSeq 2500 (Homo sapiens)', 'SDY1381': 'Illumina HiSeq 2500 (Homo sapiens)'}
rna_seq_whole_blood_T_w_meta['PLATFORM_GEO_ID'] = rna_seq_whole_blood_T_w_meta['Study'].apply(lambda x: mapping_platform_geo_id.get(x))
rna_seq_whole_blood_T_w_meta['PLATFORM_DESCRIPTION'] = rna_seq_whole_blood_T_w_meta['Study'].apply(lambda x: mapping_platform_description.get(x))


## Rearrange columns: 
cols = rna_seq_whole_blood_T_w_meta.columns.tolist()
cols = cols[-6:] + cols[:-6]
rna_seq_whole_blood_T_w_meta = rna_seq_whole_blood_T_w_meta[cols]
rna_seq_whole_blood_T_w_meta = rna_seq_whole_blood_T_w_meta.drop_duplicates()
#rna_seq_whole_blood_T_w_meta = rna_seq_whole_blood_T_w_meta.dropna()
### maybe 10k uses other datasets besides immport and those are nan ? -> figure out
rna_seq_whole_blood_T_w_meta['Sex'] = rna_seq_whole_blood_T_w_meta['Sex'].apply(lambda x: x.lower() if isinstance(x, str) else x)

rna_seq_whole_blood_T_w_meta.to_csv(f'whole_blood/rna_seq_whole_blood/rnaseq_whole_blood_prepared.csv')
rna_seq_whole_blood_T_w_meta.head()

rna_seq_whole_blood_T_w_meta

#Studies: SDY1092, SDY1172, SDY1412 and -> missing: SDY1381
### that one was tricky, in all_Data we got a leading X infront of all subjects from SDY1381, 
### i will now add a leading X to my metadata for this study and hope it fixes it

In [ ]:
   
violinplot_overall(rna_seq_whole_blood_T_w_meta, x_topic='Sex', y_topic='Age', hue_split='Sex', bin_name='whole_blood/rna_seq_whole_blood')


In [ ]:
manhattanplot(rna_seq_whole_blood_T_w_meta, start_col=7, save_name='whole_blood/rna_seq_whole_blood')
# Große abweichung von array


# Aus paper. Leider keine info, da des wohl nachgereicht wurde:
#   Additionally, some highly sought data types, such as RNA sequencing (RNA-seq), are not yet available in sufficient volume in ImmPort to merit inclusion in the initial release of the resource

In [ ]:
rna_seq_whole_blood_T_wo_meta = rna_seq_whole_blood_T_w_meta.iloc[:,7:]
metadata = rna_seq_whole_blood_T_w_meta.iloc[:,:7]
rna_seq_whole_blood_T_wo_meta = rna_seq_whole_blood_T_wo_meta.astype(float)
rna_seq_whole_blood_T_wo_meta.head()

## Own preprocessing of RNAseq whole blood 
#### was done to "match" manhattan plots

In [ ]:
## AXIS 0 is rows (y) and AXIS 1 is cols (x)
### we do this to rteduce technical artifacty that i assume are super high 
### quantile normalization is a technique for making two distributions identical in statistical properties
# I do this using qnorm
import qnorm
rna_seq_whole_blood_T_wo_meta_quant = qnorm.quantile_normalize(rna_seq_whole_blood_T_wo_meta, axis=0)
#### -> you do this if you expect the distributions to be the same. Actually i can not ...

rna_seq_whole_blood_T_wo_meta_quant = rna_seq_whole_blood_T_wo_meta_quant+1 ## I do this bc log2 of 0-1 value is negative
rna_seq_whole_blood_T_wo_meta_log2 = np.log2(rna_seq_whole_blood_T_wo_meta_quant)
rna_seq_whole_blood_T_wo_meta_log2.to_csv('whole_blood/rna_seq_whole_blood/rna_seq_whole_blood_T_wo_meta_preproc.csv')


In [ ]:
rna_seq_whole_blood_T_w_meta = rna_seq_whole_blood_T_w_meta.rename(columns={'Age': 'age', 'Sex': 'gender', 'Study':'study_accession', 'Expression': 'data_accession', 
                                             'Subject': 'subject_accession'})
print(rna_seq_whole_blood_T_w_meta['gender'].unique())
## apply the preproc to the with meta df
rna_seq_whole_blood_T_w_meta_log2 = metadata.join(rna_seq_whole_blood_T_wo_meta_log2)
#rna_seq_whole_blood_T_w_meta_log2 = rna_seq_whole_blood_T_wo_meta_log2.combine_first(rna_seq_whole_blood_T_w_meta)
rna_seq_whole_blood_T_w_meta_log2.to_csv(f'whole_blood/rna_seq_whole_blood/rna_seq_whole_blood_T_w_meta_preproc.csv')

In [ ]:
rna_seq_whole_blood_T_w_meta_log2.shape

In [ ]:
rna_seq_whole_blood_T_w_meta_log2.describe()

In [ ]:
description = rna_seq_whole_blood_T_w_meta_log2.describe().T
negative_mins = description[description['min'] < 0]
print(negative_mins)

In [ ]:
manhattanplot(rna_seq_whole_blood_T_wo_meta_log2)

# ################# MERGE whole blood

In [ ]:
### Merge the data
## Step1: Common cols: 
## Step2: Common metadata: (adapt rna_seq_whole_blood_T_w_meta to marray)
rna_seq_whole_blood_T_w_meta_log2['METHOD']='rnaseq'
marray_whole_blood['METHOD']='array'
common_columns = rna_seq_whole_blood_T_w_meta_log2.columns.intersection(marray_whole_blood.columns)

combi_df_whole_blood = pd.concat([marray_whole_blood, rna_seq_whole_blood_T_w_meta_log2], ignore_index=True)
combi_df_whole_blood['gender'] = combi_df_whole_blood['gender'].str.lower()
combi_df_whole_blood['TYPE']='whole_blood'
### Combi df of all data CONTAINING Nan and possible duplicates
print(combi_df_whole_blood.shape)
#combi_df_whole_blood.to_csv(f'whole_blood/whole_blood_combi_all.csv')

In [ ]:
no_meta_cols = list(set(common_columns) - set(METADATA_COLS))
print(len(no_meta_cols))
new_order_cols = METADATA_COLS + no_meta_cols
print(len(new_order_cols))

In [ ]:
combi_df_whole_blood_common_columns = combi_df_whole_blood[new_order_cols]
combi_df_whole_blood_common_columns.to_csv(f'whole_blood/whole_blood_combi_common_columns.csv')

In [ ]:
combi_df_whole_blood_common_columns.head()

In [ ]:
combi_df_whole_blood.head()

In [ ]:

### Remove possible duplicated subjects and entries with Nan 
combi_df_whole_blood_reduced = combi_df_whole_blood.drop_duplicates()
combi_df_whole_blood_reduced = combi_df_whole_blood_reduced.drop_duplicates(subset=['subject_accession'], keep='first')
print(combi_df_whole_blood_reduced.shape)

## drop entries that dont fullfill those min metadata
essential_columns = ['age', 'gender', 'PLATFORM_GEO_ID']
combi_df_whole_blood_reduced = combi_df_whole_blood_reduced.dropna(subset=essential_columns, axis=0, how='any')
print(combi_df_whole_blood_reduced.shape)
### Here i wanted to fix an error that i made by renaming the cols (sex, gender... )

# TODO 
## impute mean value 
## calc percentage of nan per gene. 
## drop biomarkers for values with Nan
# this reduces the genes to about /2 
combi_df_whole_blood_reduced = combi_df_whole_blood_reduced.dropna(axis=1, how='any') 
print(combi_df_whole_blood_reduced.shape)
combi_df_whole_blood_reduced.to_csv(f'whole_blood/whole_blood_combi_reduced.csv')

In [ ]:
violinplot_overall(combi_df_whole_blood_reduced,  x_topic='gender', y_topic='age', hue_split='gender', bin_name='whole_blood')

In [ ]:
combi_df_whole_blood_reduced.columns

In [ ]:
manhattanplot(combi_df_whole_blood_reduced, start_col=9, save_name='whole_blood/manhattan_plot.csv')

### individual plots

In [ ]:

### So we got our tables now: 
##### Next steps
# TODO: 
## - find paper that uses microarray and RNAseq, both...
#### decided to wait for answer of 10k --- 
## - find nonhealthy samples (what are  the samples we currently got ?) -> they are healthy as 10k lists only healthy subjects! 
#### "Subject-level immunology data from >10,000 healthy normal human subjects"



In [ ]:
description = combi_df_whole_blood_reduced.describe().T
negative_mins = description[description['min'] < 0]
print(negative_mins)
print(combi_df_whole_blood_reduced.shape)
combi_df_whole_blood_reduced.describe()
